# Actual Backtest

In [ ]:
import pandas as pd
import talib
import numpy as np
from binance.client import Client
import matplotlib.pyplot as plt

# Initialize the Binance client
api_key = 'your_api'
api_secret = 'your_secret'
client = Client(api_key, api_secret)

# Function to fetch historical data over 6 months in chunks
def get_full_historical_klines(symbol, interval, start_str, end_str):
    data = []
    klines = client.get_historical_klines(symbol, interval, start_str=start_str, end_str=end_str)
    data.extend(klines)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Fetch historical data with a 15-minute interval for 6 months
symbol = 'ETHUSDT'
interval = '2h'
start_str = '1st June 2024 UTC'
end_str = None #'1st September 2024 UTC'

df = get_full_historical_klines(symbol, interval, start_str,end_str)

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)

# Initialize variables for backtesting
position = None
initial_balance = 10000  # Starting with $10,000
balance = initial_balance
qty = 0.3  # Number of units to trade
holdings = 0
entry_price = 0
trade_log = []
balance_over_time = []


# Backtesting loop
for i in range(1, len(df)):
    current_price = df['close'].iloc[i]
    rsi = df['RSI'].iloc[i]
    timestamp = df['timestamp'].iloc[i]

    # Record balance over time
    balance_over_time.append({'timestamp': timestamp, 'balance': balance})

    # Check if no position is currently held
    if position is None:
#         Long position entry condition
        if rsi > 70:
            position = 'LONG'
            entry_price = current_price
            holdings = qty
            balance -= entry_price * qty
            trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': position})

        # Short position entry condition
        elif rsi < 30:
            position = 'SHORT'
            entry_price = current_price
            holdings = qty
            balance += entry_price * qty
            trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': position})

    # If in a LONG position, check for exit condition
    elif position == 'LONG' and rsi < 70:
        balance += current_price * holdings
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
        position = None

    # If in a SHORT position, check for exit condition
    elif position == 'SHORT' and rsi > 30:
        balance -= current_price * holdings
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
        position = None

# Final balance calculation (accounting for any remaining positions)
if position == 'LONG':
    balance += current_price * holdings
elif position == 'SHORT':
    balance -= current_price * holdings

# Output results
print(f"Initial Balance: ${initial_balance}")
print(f"Final Balance: ${balance}")
print(f"Net Profit/Loss: ${balance - initial_balance}")

# Convert trade log to DataFrame for analysis
trade_log_df = pd.DataFrame(trade_log)

# Output the trade log
print(trade_log_df)

# Save trade log to CSV
trade_log_df.to_csv('eth_data.csv', index=False)

# Convert balance over time to DataFrame
balance_over_time_df = pd.DataFrame(balance_over_time)

# Bot for LONG

In [ ]:
import pandas as pd
import talib
import numpy as np
from binance.client import Client
import time
from binance.enums import SIDE_BUY, SIDE_SELL, ORDER_TYPE_MARKET

# Initialize the Binance client
api_key = 'your_api'
api_secret = 'your_secret'
client = Client(api_key, api_secret)

# Function to fetch recent data (e.g., for the last 200 candlesticks)
def fetch_recent_data(symbol, interval):
    try:
        klines = client.get_klines(symbol=symbol, interval=interval, limit=200)
        df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                           'close_time', 'quote_asset_volume', 'number_of_trades',
                                           'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        df['close'] = df['close'].astype(float)
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

# Function to place an order on Binance
def place_order(symbol, side, quantity, order_type=ORDER_TYPE_MARKET):
    try:
        print(f"Placing {side} order for {quantity} {symbol}")
        order = client.create_order(
            symbol=symbol,
            side=side,
            type=order_type,
            quantity=quantity
        )
        print(order)
    except Exception as e:
        print(f"An exception occurred: {e}")
        return None
    return order

# Bot logic to run continuously
def run_trading_bot():
    symbol = 'ETHUSDT'
    interval = '15m'  # 15-minute candles
    position = None
    balance = 1000  # Starting balance in USDT
    qty = 0.3  # Number of ETH to trade
    entry_price = 0
    holdings = 0
    trade_log = []

    while True:
        # Fetch the latest data
        df = fetch_recent_data(symbol, interval)
        if df is None:
            time.sleep(60)  # Wait 1 minute before retrying
            continue

        # Calculate the RSI indicator
        df['RSI'] = talib.RSI(df['close'], timeperiod=14)
        current_price = df['close'].iloc[-1]  # Latest close price
        rsi = df['RSI'].iloc[-1]  # Latest RSI value
        timestamp = df['timestamp'].iloc[-1]  # Latest timestamp

        print(f"Current price: {current_price}, RSI: {rsi}")

        # Check if no position is currently held
        if position is None:
            # Long position entry condition
            if rsi > 70:
                position = 'LONG'
                entry_price = current_price
                holdings = qty
                balance -= entry_price * qty
                trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': position})
                
                # Place a buy order
                place_order(symbol, SIDE_BUY, qty)

        # If in a LONG position, check for exit condition
        elif position == 'LONG' and rsi < 70:
            balance += current_price * holdings
            holdings = 0
            trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
            
            # Place a sell order
            place_order(symbol, SIDE_SELL, qty)
            
            position = None

        # Print the latest balance and position status
        print(f"Balance: {balance}, Position: {position}")

        # Save trade log to CSV periodically
        if len(trade_log) > 0 and len(trade_log) % 5 == 0:
            trade_log_df = pd.DataFrame(trade_log)
            trade_log_df.to_csv('eth_trade_log.csv', index=False)

        # Wait before fetching new data
        time.sleep(900)  # 15-minute intervals

# Start the bot
if __name__ == "__main__":
    run_trading_bot()


# Backtest with Fees

In [ ]:
import pandas as pd
import talib
import numpy as np
from binance.client import Client
import matplotlib.pyplot as plt

# Initialize the Binance client
api_key = 'your_api'
api_secret = 'your_secret'
client = Client(api_key, api_secret)

# Function to fetch historical data over 6 months in chunks
def get_full_historical_klines(symbol, interval, start_str, end_str):
    data = []
    klines = client.get_historical_klines(symbol, interval, start_str=start_str, end_str=end_str)
    data.extend(klines)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Fetch historical data with a 15-minute interval for 6 months
symbol = 'BTCUSDT'
interval = '2h'
start_str = '1st June 2024 UTC'
end_str = None #'1st September 2024 UTC'

df = get_full_historical_klines(symbol, interval, start_str, end_str)

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)

# Initialize variables for backtesting
position = None
initial_balance = 10000  # Starting with $10,000
balance = initial_balance
qty = 0.2  # Number of units to trade
holdings = 0
entry_price = 0
trade_log = []
balance_over_time = []

# Set the trading fee (0.1%)
fee_percentage = 0.001

# Backtesting loop
for i in range(1, len(df)):
    current_price = df['close'].iloc[i]
    rsi = df['RSI'].iloc[i]
    timestamp = df['timestamp'].iloc[i]

    # Record balance over time
    balance_over_time.append({'timestamp': timestamp, 'balance': balance})

    # Check if no position is currently held
    if position is None:
        # Long position entry condition
        if rsi > 70:
            position = 'LONG'
            entry_price = current_price
            holdings = qty
            # Deduct the amount for buying, including the fee
            total_cost = entry_price * qty
            fee = total_cost * fee_percentage
            balance -= (total_cost + fee)
            trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': position, 'fee': fee})

        # Short position entry condition
        elif rsi < 30:
            position = 'SHORT'
            entry_price = current_price
            holdings = qty
            # Add the amount for selling, including the fee
            total_proceeds = entry_price * qty
            fee = total_proceeds * fee_percentage
            balance += (total_proceeds - fee)
            trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': position, 'fee': fee})

    # If in a LONG position, check for exit condition
    elif position == 'LONG' and rsi < 70:
        # Add the amount for selling, including the fee
        total_proceeds = current_price * holdings
        fee = total_proceeds * fee_percentage
        balance += (total_proceeds - fee)
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': 'EXIT', 'fee': fee})
        position = None

    # If in a SHORT position, check for exit condition
    elif position == 'SHORT' and rsi > 30:
        # Deduct the amount for buying back, including the fee
        total_cost = current_price * holdings
        fee = total_cost * fee_percentage
        balance -= (total_cost + fee)
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': 'EXIT', 'fee': fee})
        position = None

# Final balance calculation (accounting for any remaining positions)
if position == 'LONG':
    total_proceeds = current_price * holdings
    fee = total_proceeds * fee_percentage
    balance += (total_proceeds - fee)
elif position == 'SHORT':
    total_cost = current_price * holdings
    fee = total_cost * fee_percentage
    balance -= (total_cost + fee)

# Output results
print(f"Initial Balance: ${initial_balance}")
print(f"Final Balance: ${balance}")
print(f"Net Profit/Loss: ${balance - initial_balance}")

# Convert trade log to DataFrame for analysis
trade_log_df = pd.DataFrame(trade_log)

# Output the trade log
print(trade_log_df)

# Save trade log to CSV
trade_log_df.to_csv('btc_logs_with_fees.csv', index=False)

# Convert balance over time to DataFrame
balance_over_time_df = pd.DataFrame(balance_over_time)


In [ ]:
import pandas as pd
import talib
import numpy as np
from binance.client import Client
import matplotlib.pyplot as plt

# Initialize the Binance client
api_key = 'your_api'
api_secret = 'your_secret'
client = Client(api_key, api_secret)

# Function to fetch historical data over 6 months in chunks
def get_full_historical_klines(symbol, interval, start_str, end_str):
    data = []
    klines = client.get_historical_klines(symbol, interval, start_str=start_str, end_str=end_str)
    data.extend(klines)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Fetch historical data with a 15-minute interval for 6 months
symbol = 'ETHUSDT'
interval = '2h'
start_str = '1st June 2024 UTC'
end_str = None #'1st September 2024 UTC'

df = get_full_historical_klines(symbol, interval, start_str, end_str)

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)

# Initialize variables for backtesting
position = None
initial_balance = 1000  # Starting with $1,000
balance = initial_balance
qty = 0.3  # Number of units to trade
holdings = 0
entry_price = 0
trade_log = []
balance_over_time = []

# Set the trading fee (0.1%)
fee_percentage = 0.001

# Backtesting loop
for i in range(1, len(df)):
    current_price = df['close'].iloc[i]
    rsi = df['RSI'].iloc[i]
    timestamp = df['timestamp'].iloc[i]

    # Record balance over time
    balance_over_time.append({'timestamp': timestamp, 'balance': balance})

    # Check if no position is currently held
    if position is None:
        # Long position entry condition (buy ETH)
        if rsi > 70:
            position = 'LONG'
            entry_price = current_price
            holdings = qty
            # Fee is charged in the asset you receive (ETH), so we reduce the amount of ETH received
            eth_received = qty * (1 - fee_percentage)  # ETH after fee
            balance -= entry_price * eth_received  # Deduct the cost from the balance
            trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 
                              'balance': balance, 'position': position, 'fee_asset': 'ETH', 
                              'fee_amount': qty * fee_percentage})

        # Short position entry condition (sell ETH)
        elif rsi < 30:
            position = 'SHORT'
            entry_price = current_price
            holdings = qty
            # Fee is charged in USDC (or the quote asset), so we reduce the USDC received
            usdc_received = (entry_price * qty) * (1 - fee_percentage)  # USDC after fee
            balance += usdc_received  # Add the proceeds to the balance
            trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 
                              'balance': balance, 'position': position, 'fee_asset': 'USDC', 
                              'fee_amount': (entry_price * qty) * fee_percentage})

    # If in a LONG position, check for exit condition (sell ETH)
    elif position == 'LONG' and rsi < 70:
        # Fee is charged in USDC (or the quote asset), so we reduce the USDC received
        usdc_received = (current_price * holdings) * (1 - fee_percentage)  # USDC after fee
        balance += usdc_received  # Add the proceeds to the balance
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 
                          'balance': balance, 'position': 'EXIT', 'fee_asset': 'USDC', 
                          'fee_amount': (current_price * holdings) * fee_percentage})
        position = None

    # If in a SHORT position, check for exit condition (buy back ETH)
    elif position == 'SHORT' and rsi > 30:
        # Fee is charged in the asset you receive (ETH), so we reduce the amount of ETH bought back
        eth_bought_back = holdings * (1 - fee_percentage)  # ETH bought back after fee
        total_cost = current_price * eth_bought_back
        balance -= total_cost  # Deduct the cost from the balance
        holdings = 0
        trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 
                          'balance': balance, 'position': 'EXIT', 'fee_asset': 'ETH', 
                          'fee_amount': holdings * fee_percentage})
        position = None

# Final balance calculation (accounting for any remaining positions)
if position == 'LONG':
    usdc_received = (current_price * holdings) * (1 - fee_percentage)
    balance += usdc_received
elif position == 'SHORT':
    eth_bought_back = holdings * (1 - fee_percentage)
    balance -= current_price * eth_bought_back

# Output results
print(f"Initial Balance: ${initial_balance}")
print(f"Final Balance: ${balance}")
print(f"Net Profit/Loss: ${balance - initial_balance}")

# Convert trade log to DataFrame for analysis
trade_log_df = pd.DataFrame(trade_log)

# Output the trade log
print(trade_log_df)

# Save trade log to CSV
trade_log_df.to_csv('eth_logs_with_fees.csv', index=False)

# Convert balance over time to DataFrame
balance_over_time_df = pd.DataFrame(balance_over_time)


In [ ]:
#Monthly Profits

import pandas as pd

# Assuming the DataFrame `df` is already loaded with your data
df = pd.read_csv('trade_log.csv')

# Filter the DataFrame to get only the rows where position is 'EXIT'
exit_positions = df[df['position'] == 'EXIT'].copy()

# Convert the 'timestamp' column to datetime if it's not already
exit_positions['timestamp'] = pd.to_datetime(exit_positions['timestamp'])

# Extract the year and month for grouping
exit_positions['year_month'] = exit_positions['timestamp'].dt.to_period('M')

# Create a column to track the previous balance
exit_positions['prev_balance'] = exit_positions['balance'].shift(1)

# Calculate profit/loss for each trade
exit_positions['profit_loss'] = exit_positions['balance'] - exit_positions['prev_balance']

# For the first EXIT position, use the initial balance of 10000 for calculating profit/loss
initial_balance = 10000
exit_positions.iloc[0, exit_positions.columns.get_loc('profit_loss')] = exit_positions.iloc[0]['balance'] - initial_balance

# Determine whether each trade was a win or a loss
exit_positions['result'] = exit_positions['profit_loss'] > 0

# Group by year_month and calculate win rate and total profit/loss
monthly_stats = exit_positions.groupby('year_month').agg(
    win_rate=('result', 'mean'),
    total_trades=('result', 'count'),
    total_profit_loss=('profit_loss', 'sum')  # Sum of profit/loss for the month
).reset_index()

# Convert win rate to percentage
monthly_stats['win_rate'] = monthly_stats['win_rate'] * 100

# Display the results
print(monthly_stats)


# RSI Variations BackTest

In [ ]:
import pandas as pd
import talib
import numpy as np
from binance.client import Client
import matplotlib.pyplot as plt

# Initialize the Binance client
api_key = 'your_api'
api_secret = 'your_secret'
client = Client(api_key, api_secret)

def get_full_historical_klines(symbol, interval, start_str, end_str):
    data = []
    klines = client.get_historical_klines(symbol, interval, start_str=start_str, end_str=end_str)
    data.extend(klines)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Fetch historical data
symbol = 'BNBUSDT'
interval = '2h'
start_str = '1st June 2024 UTC'#6 months ago UTC'#
end_str = '1st September 2024 UTC'

df = get_full_historical_klines(symbol, interval, start_str, end_str)

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=10)

# Initialize variables for backtesting
initial_balance = 10000  # Starting with $10,000
qty = 17  # Number of units to trade

# List to store results
results = []

# Iterate over the range of RSI Entry and Exit values
for rsi_entry in range(76, 59, -1):
    for rsi_exit in range(76, 59, -1):
        balance = initial_balance
        position = None
        holdings = 0
        entry_price = 0
        trade_log = []
        balance_over_time = []

        # Backtesting loop
        for i in range(1, len(df)):
            current_price = df['close'].iloc[i]
            rsi = df['RSI'].iloc[i]
            timestamp = df['timestamp'].iloc[i]

            # Record balance over time
            balance_over_time.append({'timestamp': timestamp, 'balance': balance})

            # Check if no position is currently held
            if position is None:
                # Long position entry condition
                if rsi > rsi_entry:
                    position = 'LONG'
                    entry_price = current_price
                    holdings = qty
                    balance -= entry_price * qty
                    trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': position})

            # If in a LONG position, check for exit condition
            elif position == 'LONG' and rsi < rsi_exit:
                balance += current_price * holdings
                holdings = 0
                trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
                position = None
        
        # Final balance calculation (accounting for any remaining positions)
        if position == 'LONG':
            balance += current_price * holdings

        # Calculate net profit/loss percentage
        net_profit_loss = (balance - initial_balance) / initial_balance * 100

        # Store the result
        results.append({
            'RSI Entry': rsi_entry,
            'RSI Exit': rsi_exit,
            'Net Profit/Loss (%)': net_profit_loss
        })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Pivot the DataFrame to create the desired table
pivot_table = results_df.pivot(index='RSI Exit', columns='RSI Entry', values='Net Profit/Loss (%)')

# Display the table
print(pivot_table)

# Save trade log to CSV
pivot_table.to_csv('pivot_BNB2h_LONG_RSI10_3m_12.csv', index=False)

# Win Rate + Visualizations of wins/losses

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming the DataFrame `df` is already provided as in your example
# Load the data
df = pd.read_csv('btc_data.csv')
# Initialize the initial balance
initial_balance = 10000

# Filter the DataFrame to get only the rows where position is 'EXIT'
exit_positions = df[df['position'] == 'EXIT'].copy()

# Create a new column to store the previous balance for the comparisons
exit_positions['prev_balance'] = exit_positions['balance'].shift(1)

# Manually set the previous balance for the first trade to the initial balance
exit_positions.iloc[0, exit_positions.columns.get_loc('prev_balance')] = initial_balance

# Plot the balance amounts for the EXIT positions with color coding for wins and losses
plt.figure(figsize=(10, 6))

# Plot the points with color coding
for i, row in exit_positions.iterrows():
    if row['balance'] > row['prev_balance']:
        plt.plot(row['timestamp'], row['balance'], 'go-', marker='o', linestyle='-')  # Green for wins
    else:
        plt.plot(row['timestamp'], row['balance'], 'ro-', marker='o', linestyle='-')  # Red for losses

# Set plot details
plt.title('ETHUSDT Balance Amount When Position is EXIT')
plt.xlabel('Timestamp')
plt.ylabel('Balance')
plt.xticks(rotation=90)  # Rotate the x-axis labels for better readability
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()

# Calculate win rate
total_trades = len(exit_positions)
wins = len(exit_positions[exit_positions['balance'] > exit_positions['prev_balance']])
win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0
losses = total_trades - wins
loss_rate = (losses / total_trades) * 100 if total_trades > 0 else 0

print(f'Win Rate: {win_rate:.2f}%')
print(f'Loss Rate: {loss_rate:.2f}%')

# Output the details of each trade for verification
for i, row in exit_positions.iterrows():
    print(f"Trade {i}: Previous Balance = {row['prev_balance']}, Current Balance = {row['balance']}, {'Win' if row['balance'] > row['prev_balance'] else 'Loss'}")

## Profit/Loss per Month

In [ ]:
df = pd.read_csv('trade_log_with_fees.csv')

# Filter the DataFrame to get only the rows where position is 'EXIT'
exit_positions = df[df['position'] == 'EXIT'].copy()

# Convert the 'timestamp' column to datetime if it's not already
exit_positions['timestamp'] = pd.to_datetime(exit_positions['timestamp'])

# Extract the year and month for grouping
exit_positions['year_month'] = exit_positions['timestamp'].dt.to_period('M')

# Create a column to track the previous balance
exit_positions['prev_balance'] = exit_positions['balance'].shift(1)

# Calculate profit/loss for each trade
exit_positions['profit_loss'] = exit_positions['balance'] - exit_positions['prev_balance']

# For the first EXIT position, use the initial balance of 10000 for calculating profit/loss
initial_balance = 10000
exit_positions.iloc[0, exit_positions.columns.get_loc('profit_loss')] = exit_positions.iloc[0]['balance'] - initial_balance

# Determine whether each trade was a win or a loss
exit_positions['result'] = exit_positions['profit_loss'] > 0

# Group by year_month and calculate win rate and total profit/loss
monthly_stats = exit_positions.groupby('year_month').agg(
    win_rate=('result', 'mean'),
    total_trades=('result', 'count'),
    total_profit_loss=('profit_loss', 'sum')  # Sum of profit/loss for the month
).reset_index()

# Convert win rate to percentage
monthly_stats['win_rate'] = monthly_stats['win_rate'] * 100

# Display the results
print(monthly_stats)

# Visualizations

In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Load the BTCUSDT and ETHUSDT data from the CSV files
btc_data = pd.read_csv('btc_data.csv')
eth_data = pd.read_csv('eth_data.csv')

# Filter the data for 'EXIT' positions
btc_exit_positions = btc_data[btc_data['position'] == 'EXIT']
eth_exit_positions = eth_data[eth_data['position'] == 'EXIT']

# Convert the timestamps to pandas datetime format for easier merging
btc_exit_positions['timestamp'] = pd.to_datetime(btc_exit_positions['timestamp'])
eth_exit_positions['timestamp'] = pd.to_datetime(eth_exit_positions['timestamp'])

# Merge the datasets on the 'timestamp' column using an outer join to include all timestamps
# merged_data = pd.merge_asof(btc_exit_positions.sort_values('timestamp'),
#                             eth_exit_positions.sort_values('timestamp'),
#                             on='timestamp',
#                             direction='nearest',  # Match the nearest timestamp between the two datasets
#                             suffixes=('_btc', '_eth'))

# Create the Plotly figure
fig = go.Figure()

# Plot BTCUSDT balance progression
# fig.add_trace(go.Scatter(x=merged_data['timestamp'],
#                          y=merged_data['balance_btc'],
#                          mode='lines+markers',
#                          name='BTCUSDT Balance',
#                          line=dict(color='blue')))

# Plot ETHUSDT balance progression
fig.add_trace(go.Scatter(x=merged_data['timestamp'],
                         y=merged_data['balance_eth'],
                         mode='lines+markers',
                         name='ETHUSDT Balance',
                         line=dict(color='green')))

# Customize layout
fig.update_layout(
    title='ETH-USDT Balance Progression',
    xaxis_title='Timestamp',
    yaxis_title='Balance',
    xaxis=dict(tickangle=-45),
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.5)', bordercolor='black', borderwidth=1),
    hovermode='x unified',
    template='plotly_white',
    margin=dict(l=40, r=40, t=50, b=50)
)

# Show the figure
fig.show()

In [ ]:
# Load the ETHUSDT data from the CSV file
eth_data = pd.read_csv('eth_data.csv')

# Check column names to verify the correct name
print(eth_data.columns)

# Assuming the correct balance column is 'balance', adjust the code accordingly
# Filter the data for 'EXIT' positions
eth_exit_positions = eth_data[eth_data['position'] == 'EXIT']

# Convert the timestamps to pandas datetime format for easier merging
eth_exit_positions['timestamp'] = pd.to_datetime(eth_exit_positions['timestamp'])

# Create the Plotly figure
fig = go.Figure()

# Replace 'balance_eth' with the correct column name for balance, for example 'balance'
fig.add_trace(go.Scatter(x=eth_exit_positions['timestamp'],
                         y=eth_exit_positions['balance'],  # Adjust this if balance column has a different name
                         mode='lines+markers',
                         name='ETHUSDT Balance',
                         line=dict(color='green')))

# Customize layout
fig.update_layout(
    title='ETH-USDT Balance Progression',
    xaxis_title='Timestamp',
    yaxis_title='Balance',
    xaxis=dict(tickangle=-45),
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255, 255, 255, 0.5)', bordercolor='black', borderwidth=1),
    hovermode='x unified',
    template='plotly_white',
    margin=dict(l=40, r=40, t=50, b=50)
)

# Show the figure
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Load the BTCUSDT and ETHUSDT data from the CSV files
btc_data = pd.read_csv('btc_data.csv')
eth_data = pd.read_csv('eth_data.csv')

# Filter the data for 'EXIT' positions
btc_exit_positions = btc_data[btc_data['position'] == 'EXIT'].copy()
eth_exit_positions = eth_data[eth_data['position'] == 'EXIT'].copy()

# Convert the timestamps to pandas datetime format for easier merging
btc_exit_positions['timestamp'] = pd.to_datetime(btc_exit_positions['timestamp'])
eth_exit_positions['timestamp'] = pd.to_datetime(eth_exit_positions['timestamp'])

# Merge the datasets on the 'timestamp' column using an outer join to include all timestamps
merged_data = pd.merge_asof(btc_exit_positions.sort_values('timestamp'),
                            eth_exit_positions.sort_values('timestamp'),
                            on='timestamp',
                            direction='nearest',  # Match the nearest timestamp between the two datasets
                            suffixes=('_btc', '_eth'))

# Calculate win rate, average win, average loss, and total trades
initial_balance = 10000

# For BTC
btc_exit_positions['prev_balance'] = btc_exit_positions['balance'].shift(1)
btc_exit_positions['profit_loss'] = btc_exit_positions['balance'] - btc_exit_positions['prev_balance']
btc_exit_positions.iloc[0, btc_exit_positions.columns.get_loc('profit_loss')] = btc_exit_positions.iloc[0]['balance'] - initial_balance
btc_exit_positions['result'] = btc_exit_positions['profit_loss'] > 0

# Calculate statistics for BTC
btc_avg_win = btc_exit_positions[btc_exit_positions['result']]['profit_loss'].mean()
btc_avg_loss = btc_exit_positions[~btc_exit_positions['result']]['profit_loss'].mean()
btc_win_rate = btc_exit_positions['result'].mean() * 100
btc_num_trades = len(btc_exit_positions)

# Create the Plotly figure
fig = go.Figure()

# Plot BTCUSDT balance progression
# fig.add_trace(go.Scatter(x=merged_data['timestamp'],
#                          y=merged_data['balance_btc'],
#                          mode='lines+markers',
#                          name='BTCUSDT Balance',
#                          line=dict(color='blue')))

# Plot ETHUSDT balance progression
fig.add_trace(go.Scatter(x=merged_data['timestamp'],
                         y=merged_data['balance_eth'],
                         mode='lines+markers',
                         name='ETHUSDT Balance',
                         line=dict(color='green')))

# Add a custom legend with the computed statistics
# legend_text = (f"BTC Stats:<br>"
#                f"Avg Win: {btc_avg_win:.2f}<br>"
#                f"Avg Loss: {btc_avg_loss:.2f}<br>"
#                f"Win Rate: {btc_win_rate:.2f}%<br>"
#                f"# Trades: {btc_num_trades}")

# Customize layout to add the text box for the stats
fig.update_layout(
    title='ETH-USDT Balance Progression',
    xaxis_title='Timestamp',
    yaxis_title='Balance',
    xaxis=dict(tickangle=-45),
    hovermode='x unified',
    template='plotly_white',
    margin=dict(l=40, r=40, t=50, b=50)
)

# Show the figure
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Load the BTCUSDT and ETHUSDT data from the CSV files
btc_data = pd.read_csv('btc_data.csv')
eth_data = pd.read_csv('eth_data.csv')

# Filter the data for 'EXIT' positions (each exit means a trade)
btc_exit_positions = btc_data[btc_data['position'] == 'EXIT'].copy()
eth_exit_positions = eth_data[eth_data['position'] == 'EXIT'].copy()

# Convert the timestamps to pandas datetime format for easier merging
btc_exit_positions['timestamp'] = pd.to_datetime(btc_exit_positions['timestamp'])
eth_exit_positions['timestamp'] = pd.to_datetime(eth_exit_positions['timestamp'])

# Merge the datasets on the 'timestamp' column using an outer join to include all timestamps
merged_data = pd.merge_asof(btc_exit_positions.sort_values('timestamp'),
                            eth_exit_positions.sort_values('timestamp'),
                            on='timestamp',
                            direction='nearest',  # Match the nearest timestamp between the two datasets
                            suffixes=('_btc', '_eth'))

# Calculate win rate, average win, average loss, and number of trades for BTC
initial_balance_btc = 10000
btc_exit_positions['prev_balance'] = btc_exit_positions['balance'].shift(1)
btc_exit_positions['profit_loss'] = btc_exit_positions['balance'] - btc_exit_positions['prev_balance']
btc_exit_positions.iloc[0, btc_exit_positions.columns.get_loc('profit_loss')] = btc_exit_positions.iloc[0]['balance'] - initial_balance_btc
btc_exit_positions['result'] = btc_exit_positions['profit_loss'] > 0
btc_avg_win = btc_exit_positions[btc_exit_positions['result']]['profit_loss'].mean() / initial_balance_btc * 100  # Convert to percentage
btc_avg_loss = btc_exit_positions[~btc_exit_positions['result']]['profit_loss'].mean() / initial_balance_btc * 100  # Convert to percentage
btc_win_rate = btc_exit_positions['result'].mean() * 100
btc_num_trades = btc_exit_positions.shape[0]  # Number of 'EXIT' positions (trades)

# Calculate win rate, average win, average loss, and number of trades for ETH
initial_balance_eth = 10000
eth_exit_positions['prev_balance'] = eth_exit_positions['balance'].shift(1)
eth_exit_positions['profit_loss'] = eth_exit_positions['balance'] - eth_exit_positions['prev_balance']
eth_exit_positions.iloc[0, eth_exit_positions.columns.get_loc('profit_loss')] = eth_exit_positions.iloc[0]['balance'] - initial_balance_eth
eth_exit_positions['result'] = eth_exit_positions['profit_loss'] > 0
eth_avg_win = eth_exit_positions[eth_exit_positions['result']]['profit_loss'].mean() / initial_balance_eth * 100  # Convert to percentage
eth_avg_loss = eth_exit_positions[~eth_exit_positions['result']]['profit_loss'].mean() / initial_balance_eth * 100  # Convert to percentage
eth_win_rate = eth_exit_positions['result'].mean() * 100
eth_num_trades = eth_exit_positions.shape[0]  # Number of 'EXIT' positions (trades)

# Create the Plotly figure
fig = go.Figure()

# Plot BTCUSDT balance progression
fig.add_trace(go.Scatter(x=merged_data['timestamp'],
                         y=merged_data['balance_btc'],
                         mode='lines+markers',
                         name='BTCUSDT Balance',
                         line=dict(color='blue')))

# Plot ETHUSDT balance progression
fig.add_trace(go.Scatter(x=merged_data['timestamp'],
                         y=merged_data['balance_eth'],
                         mode='lines+markers',
                         name='ETHUSDT Balance',
                         line=dict(color='green')))

# Add a custom legend with the computed statistics for both BTC and ETH
legend_text = (
    f"BTC Stats:<br>"
    f"Avg Win: {btc_avg_win:.2f}%<br>"
    f"Avg Loss: {btc_avg_loss:.2f}%<br>"
    f"Win Rate: {btc_win_rate:.2f}%<br>"
    f"# Trades: {btc_num_trades}<br><br>"
    f"ETH Stats:<br>"
    f"Avg Win: {eth_avg_win:.2f}%<br>"
    f"Avg Loss: {eth_avg_loss:.2f}%<br>"
    f"Win Rate: {eth_win_rate:.2f}%<br>"
    f"# Trades: {eth_num_trades}"
)

# Customize layout to add the text box for the stats with a border
fig.update_layout(
    title='BTCUSDT and ETHUSDT Balance Progression',
    xaxis_title='Timestamp',
    yaxis_title='Balance',
    xaxis=dict(tickangle=-45),
    hovermode='x unified',
    template='plotly_white',
    margin=dict(l=40, r=40, t=50, b=50),
    annotations=[dict(
        text=legend_text,
        xref='paper', yref='paper',
        x=1.15, y=0.5,
        showarrow=False,
        font=dict(size=12),
        align='left',
        bordercolor='black',
        borderwidth=1,
        borderpad=10,
        bgcolor='rgba(255,255,255,0.8)'
    )]
)

# Show the figure
fig.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the BTCUSDT and ETHUSDT data from the CSV files
btc_data = pd.read_csv('btc_data.csv')
eth_data = pd.read_csv('eth_data.csv')

# Filter the data for 'EXIT' positions
btc_exit_positions = btc_data[btc_data['position'] == 'EXIT']
eth_exit_positions = eth_data[eth_data['position'] == 'EXIT']

# Convert the timestamps to pandas datetime format for easier merging
btc_exit_positions['timestamp'] = pd.to_datetime(btc_exit_positions['timestamp'])
eth_exit_positions['timestamp'] = pd.to_datetime(eth_exit_positions['timestamp'])

# Merge the datasets on the 'timestamp' column using an outer join to include all timestamps
merged_data = pd.merge_asof(btc_exit_positions.sort_values('timestamp'),
                            eth_exit_positions.sort_values('timestamp'),
                            on='timestamp',
                            direction='nearest',  # Match the nearest timestamp between the two datasets
                            suffixes=('_btc', '_eth'))

# Calculate the 50/50 investment combined balance
merged_data['combined_balance'] = (merged_data['balance_btc'] * 0.5) + (merged_data['balance_eth'] * 0.5)

# Create the plot
plt.figure(figsize=(10, 6))

# Plot BTCUSDT balance progression
# plt.plot(merged_data['timestamp'], merged_data['balance_btc'], marker='o', linestyle='-', color='b', label='BTCUSDT Balance')

# Plot ETHUSDT balance progression
# plt.plot(merged_data['timestamp'], merged_data['balance_eth'], marker='o', linestyle='-', color='g', label='ETHUSDT Balance')

# Plot combined balance progression (50% in BTCUSDT, 50% in ETHUSDT)
plt.plot(merged_data['timestamp'], merged_data['combined_balance'], marker='o', linestyle='-', color='r', label='50/50 Combined Balance')

# Customize the plot
plt.title('BTCUSDT & ETHUSDT Combined Balance Progression')
plt.xlabel('Timestamp')
plt.ylabel('Balance')
plt.xticks(rotation=90)  # Rotate the x-axis labels for better readability
plt.grid(True)
plt.tight_layout()

# Add a legend to distinguish between the three lines
# plt.legend()

# Show the plot
plt.show()


## Visualization of Balance over time

In [ ]:
df = pd.read_csv('eth_data.csv')

# Filter the DataFrame to get only the rows where position is 'EXIT'
exit_positions = df[df['position'] == 'EXIT']

# Plot the balance amounts for the EXIT positions
plt.figure(figsize=(10, 6))
plt.plot(exit_positions['timestamp'], exit_positions['balance'], marker='o', linestyle='-', color='b')
plt.title('ETHUSDT Balance Progression')
plt.xlabel('Timestamp')
plt.ylabel('Balance')
plt.xticks(rotation=90)  # Rotate the x-axis labels for better readability
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
df = pd.read_csv('eth_data.csv')

# Filter the DataFrame to get only the rows where position is 'EXIT'
exit_positions = df[df['position'] == 'EXIT']

# Plot the balance amounts for the EXIT positions
plt.figure(figsize=(10, 6))
plt.plot(exit_positions['timestamp'], exit_positions['balance'], marker='o', linestyle='-', color='b')
plt.title('BNBUSDT Balance Amount When Position is EXIT')
plt.xlabel('Timestamp')
plt.ylabel('Balance')
plt.xticks(rotation=90)  # Rotate the x-axis labels for better readability
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()

## Winrate + Profit

In [ ]:
import pandas as pd
import talib
import numpy as np
from binance.client import Client
import matplotlib.pyplot as plt

# Initialize the Binance client
api_key = 'your_api'
api_secret = 'your_secret'
client = Client(api_key, api_secret)


import pandas as pd
import talib
import numpy as np

def get_full_historical_klines(symbol, interval, start_str, end_str):
    data = []
    klines = client.get_historical_klines(symbol, interval, start_str=start_str, end_str=end_str)
    data.extend(klines)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Fetch historical data
symbol = 'ETHUSDT'
interval = '8h'
start_str = '24 months ago UTC'
end_str = None

df = get_full_historical_klines(symbol, interval, start_str, end_str)

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)

# Initialize variables for backtesting
initial_balance = 10000  # Starting with $10,000
qty = 3  # Number of units to trade

# List to store results
results = []

# Iterate over the range of RSI Entry and Exit values
for rsi_entry in range(76, 59, -1):
    for rsi_exit in range(76, 59, -1):
        balance = initial_balance
        position = None
        holdings = 0
        entry_price = 0
        trade_log = []
        balance_over_time = []

        # Backtesting loop
        for i in range(1, len(df)):
            current_price = df['close'].iloc[i]
            rsi = df['RSI'].iloc[i]
            timestamp = df['timestamp'].iloc[i]

            # Record balance over time
            balance_over_time.append({'timestamp': timestamp, 'balance': balance})

            # Check if no position is currently held
            if position is None:
                # Short position entry condition
                if rsi > rsi_entry:
                    position = 'LONG'
                    entry_price = current_price
                    holdings = qty
                    balance -= entry_price * qty
                    trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': position})

            # If in a SHORT position, check for exit condition
            elif position == 'LONG' and rsi < rsi_exit:
                balance += current_price * holdings
                holdings = 0
                trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
                position = None

        # Final balance calculation (accounting for any remaining positions)
        if position == 'LONG':
            balance += current_price * holdings

        # Calculate net profit/loss percentage
        net_profit_loss = (balance - initial_balance) / initial_balance * 100

        # Store the result
        results.append({
            'RSI Entry': rsi_entry,
            'RSI Exit': rsi_exit,
            'Net Profit/Loss (%)': net_profit_loss
        })

        # Convert trade log to DataFrame
        trade_log_df = pd.DataFrame(trade_log)

        # Calculate win rate
        if not trade_log_df.empty:
            exit_positions = trade_log_df[trade_log_df['position'] == 'EXIT'].copy()
            exit_positions['prev_balance'] = exit_positions['balance'].shift(1)
            exit_positions.iloc[0, exit_positions.columns.get_loc('prev_balance')] = initial_balance

            total_trades = len(exit_positions)
            wins = len(exit_positions[exit_positions['balance'] > exit_positions['prev_balance']])
            win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0

            # Store win rate result
            results[-1]['Win Rate (%)'] = win_rate
        else:
            results[-1]['Win Rate (%)'] = None

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Pivot the DataFrame to create the desired tables
pivot_table_profit = results_df.pivot(index='RSI Exit', columns='RSI Entry', values='Net Profit/Loss (%)')
pivot_table_win_rate = results_df.pivot(index='RSI Exit', columns='RSI Entry', values='Win Rate (%)')

# Display the tables
# print("Net Profit/Loss Pivot Table:")
# print(pivot_table_profit)

print("\nWin Rate Pivot Table:")
print(pivot_table_win_rate)

# Save pivot tables to CSV
# pivot_table_profit.to_csv('pivot_BNB2h_SHORT_profit.csv', index=False)
pivot_table_win_rate.to_csv('pivot_ETH8h_LONG_win_rate.csv', index=False)


## Winrate + Profit + Drawdown

In [ ]:
import pandas as pd
import talib
import numpy as np
from binance.client import Client
import matplotlib.pyplot as plt

# Initialize the Binance client
api_key = 'your_api'
api_secret = 'your_secret'
client = Client(api_key, api_secret)

def get_full_historical_klines(symbol, interval, start_str, end_str):
    data = []
    klines = client.get_historical_klines(symbol, interval, start_str=start_str, end_str=end_str)
    data.extend(klines)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume',
                                     'close_time', 'quote_asset_volume', 'number_of_trades',
                                     'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['close'] = df['close'].astype(float)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Fetch historical data
symbol = 'BNBUSDT'
interval = '15m'
start_str = '24 months ago UTC'
end_str = None

df = get_full_historical_klines(symbol, interval, start_str, end_str)

# Calculate indicators
df['EMA200'] = talib.EMA(df['close'], timeperiod=200)
df['EMA50'] = talib.EMA(df['close'], timeperiod=50)
df['RSI'] = talib.RSI(df['close'], timeperiod=14)

# Initialize variables for backtesting
initial_balance = 10000  # Starting with $10,000
qty = 15  # Number of units to trade

# List to store results
results = []

# Iterate over the range of RSI Entry and Exit values
for rsi_entry in range(42, 22, -1):
    for rsi_exit in range(42, 22, -1):
        balance = initial_balance
        position = None
        holdings = 0
        entry_price = 0
        trade_log = []
        balance_over_time = []

        # Backtesting loop
        for i in range(1, len(df)):
            current_price = df['close'].iloc[i]
            rsi = df['RSI'].iloc[i]
            timestamp = df['timestamp'].iloc[i]

            # Record balance over time
            balance_over_time.append({'timestamp': timestamp, 'balance': balance})

            # Check if no position is currently held
            if position is None:
                # Short position entry condition
                if rsi < rsi_entry:
                    position = 'SHORT'
                    entry_price = current_price
                    holdings = qty
                    balance += entry_price * qty
                    trade_log.append({'timestamp': timestamp, 'action': 'SELL', 'price': current_price, 'balance': balance, 'position': position})

            # If in a SHORT position, check for exit condition
            elif position == 'SHORT' and rsi > rsi_exit:
                balance -= current_price * holdings
                holdings = 0
                trade_log.append({'timestamp': timestamp, 'action': 'BUY', 'price': current_price, 'balance': balance, 'position': 'EXIT'})
                position = None

        # Final balance calculation (accounting for any remaining positions)
        if position == 'SHORT':
            balance -= current_price * holdings

        # Calculate net profit/loss percentage
        net_profit_loss = (balance - initial_balance) / initial_balance * 100

        # Calculate Drawdown
        df_balance = pd.DataFrame(balance_over_time)
        df_balance['peak'] = df_balance['balance'].cummax()
        df_balance['drawdown'] = (df_balance['balance'] - df_balance['peak']) / df_balance['peak'] * 100
        max_drawdown = df_balance['drawdown'].min()

        # Store the result
        results.append({
            'RSI Entry': rsi_entry,
            'RSI Exit': rsi_exit,
            'Net Profit/Loss (%)': net_profit_loss,
            'Max Drawdown (%)': max_drawdown
        })

        # Convert trade log to DataFrame
        trade_log_df = pd.DataFrame(trade_log)

        # Calculate win rate
        if not trade_log_df.empty:
            exit_positions = trade_log_df[trade_log_df['position'] == 'EXIT'].copy()
            exit_positions['prev_balance'] = exit_positions['balance'].shift(1)
            exit_positions.iloc[0, exit_positions.columns.get_loc('prev_balance')] = initial_balance

            total_trades = len(exit_positions)
            wins = len(exit_positions[exit_positions['balance'] > exit_positions['prev_balance']])
            win_rate = (wins / total_trades) * 100 if total_trades > 0 else 0

            # Store win rate result
            results[-1]['Win Rate (%)'] = win_rate
        else:
            results[-1]['Win Rate (%)'] = None

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Pivot the DataFrame to create the desired tables
pivot_table_profit = results_df.pivot(index='RSI Exit', columns='RSI Entry', values='Net Profit/Loss (%)')
pivot_table_win_rate = results_df.pivot(index='RSI Exit', columns='RSI Entry', values='Win Rate (%)')
pivot_table_drawdown = results_df.pivot(index='RSI Exit', columns='RSI Entry', values='Max Drawdown (%)')

# Display the tables
# print("Net Profit/Loss Pivot Table:")
# print(pivot_table_profit)

# print("\nWin Rate Pivot Table:")
# print(pivot_table_win_rate)

# print("\nMax Drawdown Pivot Table:")
# print(pivot_table_drawdown)

# Save pivot tables to CSV
pivot_table_profit.to_csv('pivot_BNB15m_SHORT_profit.csv', index=False)
pivot_table_win_rate.to_csv('pivot_BNB15m_SHORT_win_rate.csv', index=False)
pivot_table_drawdown.to_csv('pivot_BNB15m_SHORT_drawdown.csv', index=False)